In [35]:
import numpy as np
import pandas as pd
import os

In [36]:
def calculate_trend_slope(data, window_size):
    if len(data) < window_size:
        raise ValueError("데이터 길이가 구간 크기보다 작습니다.")
    # float으로 data를 우선 만든다 그 이유는 소수점으로 나올 듯 하기 때문  좀더 정확하게 계산 하기 위해서
    window_data = data.astype(float)
    # x는 window size 만큼의 배열을 생성 
    # 이렇게 생성된 배열 X는 추세 선을 적합하기 위해 사용될 데이터 포인트의 인덱스를 나타낸다
    # 추세 선의 기울기를 결정하는데 사용됩니다.
    X = np.arange(window_size)
    # 함수는 다항식을 피팅하기 위해 주어진 데이터에 대해 최소 제곱 방법을 사용하여 다항식의 계수를 계산합니다.
    coefficients = np.polyfit(X, window_data, 1)  
    slope = coefficients[0]  # 추세선의 기울기를 추출합니다.
    nan_indices = np.isnan(window_data)
    # 데이터의 절대값의 평균을 계산합니다.
    abs_mean = np.mean(np.abs(window_data)) 
    # abs_mean이 0인 경우 예외를 발생시킵니다.
    if np.any(nan_indices):
        window_data = window_data[~nan_indices]
        X = X[~nan_indices]
        coefficients = np.polyfit(X, window_data, 1)
        slope = coefficients[0]
    abs_mean = np.mean(np.abs(window_data))
    
    if abs_mean == 0:
        return None
    else:
        standardized_slope = slope / abs_mean
        return standardized_slope

def calculate_trend_slope_daily(data, window_size):
    result = {}
    # 여기가 매우 중요한 부분 우리가 원하는 column을 tolist로 생성해야 한다
    column_names = data.columns.tolist()
    # 생성후 차례차례 column으로 넘겨서 각 column마다 돌아서 하기 위해 2중 for문 사용
    for column in column_names:
        # 계속해서 list를 생성할 수 없으니 for문 column마다 초기화
        trend_slope_result = []
        # 우리가 원하는 값이 data의 값과 window size값의 차이만큼을 원하므로 이런식으로 사용
        for i in range(len(data) - window_size + 1):
            #window_data에 iloc을 사용해서 i 부터 i+ window_size 만큼 준다 그 이후 window_data의 값을 선형회기 함수에 
            # 이 값을 result에 append 하고 result [column]에 넣고 처음으로 다시 돌아간다
            window_data = data[column].iloc[i:i+window_size]
            trend_slope = calculate_trend_slope(window_data, window_size)
            trend_slope_result.append(trend_slope)
        result[column] = trend_slope_result
    return result

In [37]:
# 데이터 불러오기 우리가 원하는 값들을 불러오면됨
raw = r'Feature_data_base.xlsx'
df = pd.read_excel(raw, sheet_name='total', index_col=0)
df.index = df.index.strftime('%Y-%m-%d')

In [38]:
df.tail()

,WTI,DGS2,DGS10,TIPS,VIX,PPI,코스피,per,pbr,원달러,...,발틱운임,scfi,미 소맥 선물,미 대두 선물,항공 여객,소비자심리지수,ICT 전망 BSI,IT 월별 수출현황,SOX,메모리 수출금액
date,,,,,,,,,,,,,,,,,,,,,
2024-04-02,85.95,4.70,2.00,2.00,14.61,248.72,2753.16,20.25,20.25,1349.4,...,1714,1730.98,545.25,1174.00,12185698,100.7,98,16533210273,4886.9,5177595
2024-04-03,86.22,4.68,2.00,2.00,14.33,248.72,2706.97,19.91,19.91,1352.1,...,1711,1745.43,556.00,1182.25,12185698,100.7,98,16533210273,4903.5,5177595
2024-04-04,87.37,4.65,1.97,1.97,16.35,248.72,2742.00,20.17,20.17,1348.9,...,1669,1745.43,556.25,1180.00,12185698,100.7,98,16533210273,4756.1,5177595
2024-04-05,87.69,4.73,2.02,2.02,16.03,248.72,2714.21,19.96,19.96,1347.1,...,1628,1745.43,567.25,1185.00,12185698,100.7,98,16533210273,4819.1,5177595
2024-04-08,87.24,4.78,2.04,2.04,15.19,248.72,2717.65,19.98,19.98,1352.8,...,1594,1745.43,565.75,1181.50,12185698,100.7,98,16533210273,4825.4,5177595


In [39]:
df.head()

,WTI,DGS2,DGS10,TIPS,VIX,PPI,코스피,per,pbr,원달러,...,발틱운임,scfi,미 소맥 선물,미 대두 선물,항공 여객,소비자심리지수,ICT 전망 BSI,IT 월별 수출현황,SOX,메모리 수출금액
date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,81.74,1.01,3.77,1.43,19.35,170.8,1690.62,23.80,23.80,1154.8,...,3270,1072.58,552.63,1059.88,6210262,116.9,90,11722815985,366.4,1390919
2010-01-06,83.12,1.01,3.85,1.48,19.16,170.8,1705.32,24.01,24.01,1140.5,...,3259,1072.58,567.13,1058.13,6210262,116.9,90,11722815985,366.3,1390919
2010-01-07,82.60,1.03,3.85,1.44,19.06,170.8,1683.45,23.71,23.71,1136.4,...,3149,1106.28,556.63,1026.13,6210262,116.9,90,11722815985,362.3,1390919
2010-01-08,82.74,0.96,3.83,1.41,18.13,170.8,1695.26,23.88,23.88,1135.4,...,3140,1106.28,568.13,1019.38,6210262,116.9,90,11722815985,367.7,1390919
2010-01-11,82.54,0.95,3.85,1.47,17.55,170.8,1694.12,23.85,23.85,1130.5,...,3148,1106.28,574.25,1009.88,6210262,116.9,90,11288314689,366.6,1390919


In [40]:
df.info

<bound method DataFrame.info of               WTI  DGS2  DGS10  TIPS    VIX     PPI      코스피    per    pbr  \
date                                                                         
2010-01-05  81.74  1.01   3.77  1.43  19.35  170.80  1690.62  23.80  23.80   
2010-01-06  83.12  1.01   3.85  1.48  19.16  170.80  1705.32  24.01  24.01   
2010-01-07  82.60  1.03   3.85  1.44  19.06  170.80  1683.45  23.71  23.71   
2010-01-08  82.74  0.96   3.83  1.41  18.13  170.80  1695.26  23.88  23.88   
2010-01-11  82.54  0.95   3.85  1.47  17.55  170.80  1694.12  23.85  23.85   
...           ...   ...    ...   ...    ...     ...      ...    ...    ...   
2024-04-02  85.95  4.70   2.00  2.00  14.61  248.72  2753.16  20.25  20.25   
2024-04-03  86.22  4.68   2.00  2.00  14.33  248.72  2706.97  19.91  19.91   
2024-04-04  87.37  4.65   1.97  1.97  16.35  248.72  2742.00  20.17  20.17   
2024-04-05  87.69  4.73   2.02  2.02  16.03  248.72  2714.21  19.96  19.96   
2024-04-08  87.24  4.78   2.04  

In [41]:
window_size_20=20

result_20 = calculate_trend_slope_daily(df, window_size_20)

new_df_20 = df.iloc[window_size_20-1:]
result_df_20 = pd.DataFrame(result_20)


result_df_20.index=new_df_20.index

print(result_df_20)


                 WTI      DGS2     DGS10      TIPS       VIX           PPI  \
date                                                                         
2010-02-02 -0.006935 -0.009921 -0.003012 -0.006833  0.017654 -1.199605e-17   
2010-02-03 -0.006354 -0.008857 -0.002775 -0.006683  0.016993 -1.199605e-17   
2010-02-04 -0.006123 -0.008868 -0.002612 -0.006136  0.018869 -1.199605e-17   
2010-02-05 -0.006175 -0.008676 -0.002470 -0.005264  0.020143 -1.199605e-17   
2010-02-08 -0.005834 -0.008923 -0.002190 -0.004272  0.020425 -1.199605e-17   
...              ...       ...       ...       ...       ...           ...   
2024-04-02  0.003576  0.001204  0.003715  0.003715 -0.005584  1.285377e-03   
2024-04-03  0.003822  0.001235  0.003567  0.003567 -0.004442  1.185388e-03   
2024-04-04  0.004320  0.001162  0.003053  0.003053 -0.001148  1.057390e-03   
2024-04-05  0.004684  0.001131  0.002784  0.002784  0.001513  9.014609e-04   
2024-04-08  0.004663  0.001112  0.002501  0.002501  0.003474  7.

In [42]:
window_size_60=60
result_60 = calculate_trend_slope_daily(df, window_size_60)
new_df_60 = df.iloc[window_size_60-1:]
result_df_60 = pd.DataFrame(result_60)
result_df_60.index=new_df_60.index
print(result_df_60)


                 WTI      DGS2     DGS10      TIPS       VIX       PPI  \
date                                                                     
2010-04-01  0.001020  0.001886  0.000202  0.002811 -0.003742  0.000251   
2010-04-05  0.001237  0.002512  0.000351  0.003084 -0.004067  0.000245   
2010-04-06  0.001483  0.003055  0.000521  0.003383 -0.004470  0.000240   
2010-04-07  0.001698  0.003488  0.000649  0.003483 -0.004842  0.000233   
2010-04-08  0.001903  0.003838  0.000776  0.003575 -0.005295  0.000226   
...              ...       ...       ...       ...       ...       ...   
2024-04-02  0.002676  0.001580  0.001760  0.001760  0.000629  0.000299   
2024-04-03  0.002692  0.001587  0.001748  0.001748  0.000633  0.000335   
2024-04-04  0.002772  0.001580  0.001676  0.001676  0.000974  0.000370   
2024-04-05  0.002835  0.001616  0.001693  0.001693  0.001278  0.000405   
2024-04-08  0.002914  0.001673  0.001775  0.001775  0.001379  0.000439   

                 코스피       per       

In [43]:
merge_inner = pd.merge(df, result_df_20.add_suffix('_20'), left_index=True, right_index=True)
merge_inner = pd.merge(merge_inner, result_df_60.add_suffix('_60'), left_index=True, right_index=True)
print(merge_inner)
print(len(merge_inner.columns))

              WTI  DGS2  DGS10  TIPS    VIX     PPI      코스피    per    pbr  \
date                                                                         
2010-04-01  84.53  1.05   3.89  1.61  17.47  172.20  1719.17  24.13  24.13   
2010-04-05  86.36  1.18   4.01  1.70  17.02  172.20  1724.99  24.22  24.22   
2010-04-06  86.54  1.14   3.98  1.68  16.23  172.20  1726.09  24.24  24.24   
2010-04-07  85.64  1.06   3.89  1.55  16.62  172.20  1726.60  24.26  24.26   
2010-04-08  85.17  1.09   3.91  1.58  16.48  172.20  1733.78  24.42  24.42   
...           ...   ...    ...   ...    ...     ...      ...    ...    ...   
2024-04-02  85.95  4.70   2.00  2.00  14.61  248.72  2753.16  20.25  20.25   
2024-04-03  86.22  4.68   2.00  2.00  14.33  248.72  2706.97  19.91  19.91   
2024-04-04  87.37  4.65   1.97  1.97  16.35  248.72  2742.00  20.17  20.17   
2024-04-05  87.69  4.73   2.02  2.02  16.03  248.72  2714.21  19.96  19.96   
2024-04-08  87.24  4.78   2.04  2.04  15.19  248.72  2717.65  19

In [44]:
# base_filename = r'C:\Users\siim2\sic_project_final\sic\auto_raw_20_60_{}.xlsx'

# # 해당 디렉토리에 이미 존재하는 파일들을 확인
# existing_files = os.listdir(r'C:\Users\siim2\sic_project_final\sic')
# if existing_files is True:
#     print("already")
# else:
#     print("none")

# # 가장 최신 버전 찾기
# latest_version = 0
# for filename in existing_files:
#     if filename.startswith('result_version_') and filename.endswith('.xlsx'):
#         version_number = int(filename.split('_')[2].split('.')[0])
#         if version_number > latest_version:
#             latest_version = version_number


In [46]:
import os

# 기본 파일 이름
base_filename = "Feature_data_base_20_60_1.xlsx"

# 숫자 초기화
i = 0

# 파일 이름이 존재하지 않을 때까지 반복
while os.path.exists(f"{base_filename[:-5]}_{i}.xlsx"):
    i += 1

# 새 파일 이름 생성
new_filename = f"{base_filename[:-5]}_{i}.xlsx"

# DataFrame을 Excel 파일로 저장
merge_inner.to_excel("Feature_data_base_20_60_final.xlsx", index=True)